In [1]:
import pandas as pd
import numpy as np
import string
import pickle

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [2]:
train = pd.read_csv('trainMaster.csv')
final = pd.read_csv('dfTestDataLabelFinal.csv')

In [3]:
def preprocess(ReviewText):
    ReviewText = ReviewText.str.replace("(<br/>)", "")
    ReviewText = ReviewText.str.replace('(<a).*(>).*(</a>)', '')
    ReviewText = ReviewText.str.replace('(&amp)', '')
    ReviewText = ReviewText.str.replace('(&gt)', '')
    ReviewText = ReviewText.str.replace('(&lt)', '')
    ReviewText = ReviewText.str.replace('(\xa0)', ' ')
    ReviewText = ReviewText.str.replace('(\n)', ' ') 
    ReviewText = ReviewText.str.replace('==', ' ') 
    return ReviewText

train['comment_text'] = preprocess(train['comment_text'])
final['comment_text'] = preprocess(final['comment_text'])

# Target

In [4]:
x = train['comment_text']

categories = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

# Train Test Split

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train, test = train_test_split(train, random_state=42, test_size=0.30, shuffle=True)

In [7]:
print('OwO')

OwO


# K Fold

In [8]:
# from sklearn.model_selection import KFold

In [9]:
# K = 5
# kf = KFold(n_splits=K, shuffle=True, random_state = 42)

In [10]:
# dFold = {}
# for i in range(1,8):
#     dFold['Hasilfold{}'.format(i)] = kf.split(x,datay[i-1])

# Model

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, matthews_corrcoef, f1_score, log_loss

# Analyzer for Pipeline

In [12]:
stop = []
for words in stopwords.words('english'):
    s = [char for char in words if char not in string.punctuation]
    stop.append(''.join(s))
stop.extend(['may','also','across','among','beside','however','yet','within'])

# stemmer = SnowballStemmer("english")
# def stemming(sentence):
#     stemSentence = ""
#     for word in sentence.split():
#         stem = stemmer.stem(word)
#         stemSentence += stem
#         stemSentence += " "
#     stemSentence = stemSentence.strip()
#     return stemSentence

def process_normal(text):
    return [word for word in text.split() if word.lower() not in stop]

def process_nocase(text):
    return [word.lower() for word in text.split() if word.lower() not in stop]

def process_nopunc(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stop]

def process_nocasepunc(text):
    nocasepunc = [char for char in text if char not in string.punctuation]
    nocasepunc = ''.join(nocasepunc)
    return [word.lower() for word in nocasepunc.split() if word.lower() not in stop]

 # Vectorizer

In [13]:
vectorizer = TfidfVectorizer(analyzer=process_nocasepunc, ngram_range=(1,3))
# vectorizer = TfidfVectorizer(, analyzer='word', ngram_range=(1,3), norm='l2')

# Pipeline

In [14]:
LogReg_pipeline = Pipeline([
            ('classifier', LogisticRegression()),
        ])

def pipeline_model_training(category, dataTrain, targetTrain):
    # Training logistic regression model on train data
    return LogReg_pipeline.fit(dataTrain, targetTrain)

def pipeline_predict(category, dictPred, dictPredProba, dictHasil, dataTest, targetTest):
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(dataTest)
    dictPredProba['{}'.format(category)] = LogReg_pipeline.predict_proba(dataTest)
    dictPred[category] = prediction 
    dictHasil['{}_accuracy'.format(category)] = accuracy_score(targetTest, prediction)
    dictHasil['{}_f1score'.format(category)] = f1_score(targetTest, prediction, average = 'macro')
    dictHasil['{}_logloss'.format(category)] = log_loss(targetTest, dictPredProba[category])
    dictHasil['{}_rocauc'.format(category)] = roc_auc_score(targetTest, dictPredProba[category][:,1])
    dictHasil['{}_matthews_corrcoef'.format(category)] = matthews_corrcoef(targetTest, prediction)
    

In [15]:
print('OwO')

OwO


# Gridsearch

In [16]:
from sklearn.model_selection import GridSearchCV

param_learning = (0.12, 0.1, 0.08, 0.06)
max_features  = (10000,50000,100000,150000,)
param_grid = {'learning_rate': param_learning, 'n_estimators':param_estimator}

gs = GridSearchCV(pipe_xgb.steps[2][1],param_grid,scoring='accuracy')


gs = gs.fit(X_train, y_train)

print(gs.best_score_)
print(gs.best_params_)

# Predict Train

In [17]:
vectorizer.fit(train['comment_text'])

x_train = vectorizer.transform(train['comment_text'])
y_train = train.drop(labels = ['id','comment_text'], axis=1)
x_test = vectorizer.transform(test['comment_text'])
y_test = test.drop(labels = ['id','comment_text'], axis=1)

In [18]:
dPredict_train = {}
dPredictProba_train = {}
dResult_train = {}

for category in categories:
    pipeline_model_training(category, x_train, y_train[category])
    pipeline_predict(category, dPredict_train, dPredictProba_train, dResult_train, x_train, y_train[category])

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

## Report and Analysis

In [19]:
def bikin_report(hasil, namadf, namacol):
    listItem = []

    for category in categories:
        listItem.append([category,
                        hasil['{}_accuracy'.format(category)],
                        hasil['{}_f1score'.format(category)],
                        hasil['{}_logloss'.format(category)],
                        hasil['{}_rocauc'.format(category)],
                        hasil['{}_matthews_corrcoef'.format(category)],
                        ])

    namadf = pd.DataFrame(columns=['Category','{} Acc'.format(namacol), '{} F1 Score'.format(namacol),'{} Log Loss'.format(namacol),'{} ROC AUC'.format(namacol),'{} Matthew Corr Coef'.format(namacol),],
                         data=listItem)
    return namadf

In [20]:
dfReportTrain = bikin_report(dResult_train, 'dfReportTrain', 'Train')
dfReportTrain

,Category,Train Acc,Train F1 Score,Train Log Loss,Train ROC AUC,Train Matthew Corr Coef
0,toxic,0.959131,0.858404,0.106996,0.985883,0.740220
1,severe_toxic,0.991101,0.670356,0.024508,0.993053,0.385614
2,obscene,0.978648,0.873288,0.058444,0.993584,0.763383
3,threat,0.997153,0.585225,0.010421,0.994845,0.274744
4,insult,0.972336,0.815705,0.071747,0.988691,0.656400
5,identity_hate,0.992319,0.633893,0.024197,0.990425,0.356132


In [21]:
dfReportTrain.describe()

,Train Acc,Train F1 Score,Train Log Loss,Train ROC AUC,Train Matthew Corr Coef
count,6.000000,6.000000,6.000000,6.000000,6.000000
mean,0.981781,0.739479,0.049385,0.991080,0.529416
std,0.014465,0.124563,0.036531,0.003391,0.214881
min,0.959131,0.585225,0.010421,0.985883,0.274744
25%,0.973914,0.643009,0.024275,0.989124,0.363503
50%,0.984875,0.743030,0.041476,0.991739,0.521007
75%,0.992014,0.847730,0.068422,0.993451,0.719265
max,0.997153,0.873288,0.106996,0.994845,0.763383


In [22]:
def bikin_hasil_pred(hasil, namadf):
    listItem = []
    
    for i in range(len(hasil['toxic'])):
        listItem.append([hasil['toxic'][i],
                        hasil['severe_toxic'][i],
                        hasil['obscene'][i],
                        hasil['threat'][i],
                        hasil['threat'][i],
                        hasil['identity_hate'][i],])
        
        
        if((hasil['toxic'][i] + hasil['severe_toxic'][i] + hasil['obscene'][i] + hasil['threat'][i] + hasil['threat'][i] + hasil['identity_hate'][i]) == 0):
            listItem[i].append(1)
        else:
            listItem[i].append(0)
    

    namadf = pd.DataFrame(columns=['ToxicPred', 'SToxicPred','ObscenePred','ThreatPred','InsultPred','IdHatePred', 'cleanPred'],
                         data=listItem)
    return namadf

In [23]:
dfResultTrain = bikin_hasil_pred(dPredict_train, 'dfResultTrain')
dfResultTrain.head(8)

,ToxicPred,SToxicPred,ObscenePred,ThreatPred,InsultPred,IdHatePred,cleanPred
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1
5,0,0,0,0,0,0,1
6,0,0,0,0,0,0,1
7,1,0,1,0,0,0,0


In [24]:
def combine_predict_with_keylabel(namadf, pred, key):
    namadf = pd.DataFrame
    namadf = pd.concat([pred, key[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'clean']].reset_index(drop=True)], axis=1)
    return namadf

In [25]:
dfFinalTrain = combine_predict_with_keylabel('dfFinalTrain', dfResultTrain, y_train)

In [26]:
dfFinalTrain.head()

,ToxicPred,SToxicPred,ObscenePred,ThreatPred,InsultPred,IdHatePred,cleanPred,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean
0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [27]:
def filter_dataframe(dfResult):
    actCancer = dfResult[dfResult['clean']==0]
    actClean = dfResult[dfResult['clean']==1]
    
    predCancer = dfResult[((dfResult['ToxicPred'] == 1) | (dfResult['SToxicPred'] == 1) | (dfResult['ObscenePred'] == 1)
             | (dfResult['ThreatPred'] == 1) | (dfResult['InsultPred'] == 1) | (dfResult['IdHatePred'] == 1))]
    
    predCancerActCancer = dfResult[((dfResult['ToxicPred'] == dfResult['toxic']) & (dfResult['SToxicPred'] == dfResult['severe_toxic']) & (dfResult['ObscenePred'] == dfResult['obscene'])
             & (dfResult['ThreatPred'] == dfResult['threat']) & (dfResult['InsultPred'] == dfResult['insult']) & (dfResult['IdHatePred'] == dfResult['identity_hate'])) 
                                   & (dfResult['clean']==0)]
    
    predClean = dfResult[((dfResult['ToxicPred'] == 0) & (dfResult['SToxicPred'] == 0) & (dfResult['ObscenePred'] == 0)
             & (dfResult['ThreatPred'] == 0) & (dfResult['InsultPred'] == 0) & (dfResult['IdHatePred'] == 0))]
    
    predCleanActClean = dfResult[((dfResult['ToxicPred'] == dfResult['toxic']) & (dfResult['SToxicPred'] == dfResult['severe_toxic']) & (dfResult['ObscenePred'] == dfResult['obscene'])
             & (dfResult['ThreatPred'] == dfResult['threat']) & (dfResult['InsultPred'] == dfResult['insult']) & (dfResult['IdHatePred'] == dfResult['identity_hate'])) & (dfResult['clean']==1)]

    predTrueActTrue = dfResult[(dfResult['ToxicPred'] == dfResult['toxic']) & (dfResult['SToxicPred'] == dfResult['severe_toxic']) & (dfResult['ObscenePred'] == dfResult['obscene']) & (dfResult['ThreatPred'] == dfResult['threat']) & (dfResult['InsultPred'] == dfResult['insult']) & (dfResult['IdHatePred'] == dfResult['identity_hate'])]

    return actCancer, actClean, predCancer, predCancerActCancer, predClean, predCleanActClean, predTrueActTrue

In [28]:
actCancer_train, actClean_train, predCancer_train, predCancerActCancer_train, predClean_train, predCleanActClean_train, predTrueActTrue_train = filter_dataframe(dfFinalTrain)

In [29]:
def print_filter(actCancer, actClean, predCancer, predCancerActCancer, predClean, predCleanActClean,  predTrueActTrue):
    print('Cancer = Data yang mengandung salah satu dari toxic, severe toxic, obscene, threat, insult,  atau identity hate')
    print('\n')
    print('Total Data di Dataframe = {}'.format(len(actCancer)+len(actClean)))
    print('Banyak data Cancer = {}'.format(len(actCancer)))
    print('Banyak data Clean = {}'.format(len(actClean)))
    print('\n')
    print('Banyak Prediction Cancer yang benar = {}'.format(len(predCancerActCancer)))
    print('Percentage dari Prediction Cancer yang benar = {}%'.format(len(predCancerActCancer)/len(actCancer)*100))
    print('\n')
    print('Banyak Prediction Clean yang benar = {}'.format(len(predCleanActClean)))
    print('Percentage dari Prediction Clean yang benar = {}%'.format(len(predCleanActClean)/len(actClean)*100))
    print('\n')
    print('Banyak Prediction yang benar = {}'.format(len(predTrueActTrue)))
    print('Percentage dari Prediction yang benar = {}%'.format(len(predTrueActTrue)/(len(actCancer)+len(actClean))*100))

In [30]:
print_filter(actCancer_train, actClean_train, predCancer_train, predCancerActCancer_train, predClean_train, predCleanActClean_train, predTrueActTrue_train)

Cancer = Data yang mengandung salah satu dari toxic, severe toxic, obscene, threat, insult,  atau identity hate


Total Data di Dataframe = 111699
Banyak data Cancer = 11358
Banyak data Clean = 100341


Banyak Prediction Cancer yang benar = 1684
Percentage dari Prediction Cancer yang benar = 14.8265539707695%


Banyak Prediction Clean yang benar = 100119
Percentage dari Prediction Clean yang benar = 99.77875444733459%


Banyak Prediction yang benar = 101803
Percentage dari Prediction yang benar = 91.1404757428446%


In [31]:
actCancer = dfFinalTrain[dfFinalTrain['clean']==0]
actClean = dfFinalTrain[dfFinalTrain['clean']==1]
predCancer = dfFinalTrain[dfFinalTrain['cleanPred']==0]
predClean = dfFinalTrain[dfFinalTrain['cleanPred']==1]

len(actCancer), len(actClean), len(predCancer), len(predClean), 

(11358, 100341, 6790, 104909)

In [32]:
actCancer2 = dfFinalTrain[(dfFinalTrain['clean']==0) & ((dfFinalTrain['ToxicPred'] == dfFinalTrain['toxic']) & (dfFinalTrain['SToxicPred'] == dfFinalTrain['severe_toxic']) & (dfFinalTrain['ObscenePred'] == dfFinalTrain['obscene']) & (dfFinalTrain['ThreatPred'] == dfFinalTrain['threat']) & (dfFinalTrain['InsultPred'] == dfFinalTrain['insult']) & (dfFinalTrain['IdHatePred'] == dfFinalTrain['identity_hate']))]
actClean2 = dfFinalTrain[(dfFinalTrain['clean']==1) & ((dfFinalTrain['ToxicPred'] == dfFinalTrain['toxic']) & (dfFinalTrain['SToxicPred'] == dfFinalTrain['severe_toxic']) & (dfFinalTrain['ObscenePred'] == dfFinalTrain['obscene']) & (dfFinalTrain['ThreatPred'] == dfFinalTrain['threat']) & (dfFinalTrain['InsultPred'] == dfFinalTrain['insult']) & (dfFinalTrain['IdHatePred'] == dfFinalTrain['identity_hate']))]
predCancer2 = dfFinalTrain[(dfFinalTrain['cleanPred']==0) & ((dfFinalTrain['ToxicPred'] == dfFinalTrain['toxic']) & (dfFinalTrain['SToxicPred'] == dfFinalTrain['severe_toxic']) & (dfFinalTrain['ObscenePred'] == dfFinalTrain['obscene']) & (dfFinalTrain['ThreatPred'] == dfFinalTrain['threat']) & (dfFinalTrain['InsultPred'] == dfFinalTrain['insult']) & (dfFinalTrain['IdHatePred'] == dfFinalTrain['identity_hate']))]
predClean2 = dfFinalTrain[(dfFinalTrain['cleanPred']==1) & ((dfFinalTrain['ToxicPred'] == dfFinalTrain['toxic']) & (dfFinalTrain['SToxicPred'] == dfFinalTrain['severe_toxic']) & (dfFinalTrain['ObscenePred'] == dfFinalTrain['obscene']) & (dfFinalTrain['ThreatPred'] == dfFinalTrain['threat']) & (dfFinalTrain['InsultPred'] == dfFinalTrain['insult']) & (dfFinalTrain['IdHatePred'] == dfFinalTrain['identity_hate']))]

len(actCancer2), len(actClean2), len(predCancer2), len(predClean2), 

(1684, 100119, 1684, 100119)

In [33]:
# 1684 9674 = 11358
# 222 100119 = 100 341
# 6790 104909 101803

# Predict Test

In [34]:
dPredict_test = {}
dPredictProba_test = {}
dResult_test = {}

for category in categories:
    pipeline_predict(category, dPredict_test, dPredictProba_test, dResult_test, x_test, y_test[category])

## Report and Analysis

In [35]:
dfReportTest = bikin_report(dResult_test, 'dfReportTest', 'Test')
dfReportTest

,Category,Test Acc,Test F1 Score,Test Log Loss,Test ROC AUC,Test Matthew Corr Coef
0,toxic,0.905999,0.494113,0.393276,0.895506,0.126714
1,severe_toxic,0.989430,0.561135,0.036329,0.972267,0.169674
2,obscene,0.947401,0.511382,0.202643,0.926146,0.127995
3,threat,0.995363,0.516229,0.020491,0.937695,0.033095
4,insult,0.951182,0.516871,0.188484,0.926453,0.148030
5,identity_hate,0.991770,0.623406,0.029070,0.976265,0.325099


In [36]:
dfReportTest.describe()

,Test Acc,Test F1 Score,Test Log Loss,Test ROC AUC,Test Matthew Corr Coef
count,6.000000,6.000000,6.000000,6.000000,6.000000
mean,0.963524,0.537189,0.145049,0.939055,0.155101
std,0.035225,0.047709,0.146701,0.030697,0.095486
min,0.905999,0.494113,0.020491,0.895506,0.033095
25%,0.948347,0.512594,0.030885,0.926223,0.127034
50%,0.970306,0.516550,0.112407,0.932074,0.138012
75%,0.991185,0.550069,0.199103,0.963624,0.164263
max,0.995363,0.623406,0.393276,0.976265,0.325099


In [37]:
dfResultTest = bikin_hasil_pred(dPredict_test, 'dfResultTest')
dfResultTest.head(8)

,ToxicPred,SToxicPred,ObscenePred,ThreatPred,InsultPred,IdHatePred,cleanPred
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1
5,0,0,0,0,0,0,1
6,0,0,0,0,0,0,1
7,0,0,0,0,0,0,1


In [38]:
dfFinalTest = combine_predict_with_keylabel('dfFinalTest', dfResultTest, y_test)

In [39]:
dfFinalTest.head()

,ToxicPred,SToxicPred,ObscenePred,ThreatPred,InsultPred,IdHatePred,cleanPred,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean
0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [40]:
actCancer_test, actClean_test, predCancer_test, predCancerActCancer_test, predClean_test, predCleanActClean_test, predTrueActTrue_test = filter_dataframe(dfFinalTest)

In [41]:
print_filter(actCancer_test, actClean_test, predCancer_test, predCancerActCancer_test, predClean_test, predCleanActClean_test, predTrueActTrue_test)

Cancer = Data yang mengandung salah satu dari toxic, severe toxic, obscene, threat, insult,  atau identity hate


Total Data di Dataframe = 47872
Banyak data Cancer = 4867
Banyak data Clean = 43005


Banyak Prediction Cancer yang benar = 0
Percentage dari Prediction Cancer yang benar = 0.0%


Banyak Prediction Clean yang benar = 43001
Percentage dari Prediction Clean yang benar = 99.9906987559586%


Banyak Prediction yang benar = 43001
Percentage dari Prediction yang benar = 89.82494986631015%


# Predict Final

In [42]:
x_final = vectorizer.transform(final['comment_text'])
y_final = final.drop(labels = ['id','comment_text'], axis=1)

In [43]:
dPredict_final = {}
dPredictProba_final = {}
dResult_final = {}

for category in categories:
    pipeline_predict(category, dPredict_final, dPredictProba_final, dResult_final, x_final, y_final[category])

## Report and Analysis

In [44]:
dfReportFinal = bikin_report(dResult_final, 'dfReportFinal', 'Final')
dfReportFinal

,Category,Final Acc,Final F1 Score,Final Log Loss,Final ROC AUC,Final Matthew Corr Coef
0,toxic,0.906687,0.500149,0.373151,0.900273,0.134008
1,severe_toxic,0.992857,0.587973,0.025493,0.973747,0.186030
2,obscene,0.943402,0.518905,0.214811,0.915605,0.146652
3,threat,0.993982,0.518441,0.024113,0.940473,0.036941
4,insult,0.947654,0.523580,0.198489,0.913439,0.157967
5,identity_hate,0.989965,0.641598,0.033434,0.974756,0.350171


In [45]:
dfReportFinal.describe()

,Final Acc,Final F1 Score,Final Log Loss,Final ROC AUC,Final Matthew Corr Coef
count,6.000000,6.000000,6.000000,6.000000,6.000000
mean,0.962425,0.548441,0.144915,0.936382,0.168628
std,0.035685,0.054706,0.142218,0.032086,0.102369
min,0.906687,0.500149,0.024113,0.900273,0.036941
25%,0.944465,0.518557,0.027478,0.913980,0.137169
50%,0.968810,0.521242,0.115961,0.928039,0.152310
75%,0.992134,0.571875,0.210730,0.965428,0.179015
max,0.993982,0.641598,0.373151,0.974756,0.350171


In [46]:
dfResultFinal = bikin_hasil_pred(dPredict_final, 'dfResultFinal')
dfResultFinal.head(8)

,ToxicPred,SToxicPred,ObscenePred,ThreatPred,InsultPred,IdHatePred,cleanPred
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1
5,0,0,0,0,0,0,1
6,0,0,0,0,0,0,1
7,0,0,0,0,0,0,1


In [47]:
dfFinalFinal = combine_predict_with_keylabel('dfFinalFinal', dfResultFinal, final)

In [48]:
dfFinalFinal.head()

,ToxicPred,SToxicPred,ObscenePred,ThreatPred,InsultPred,IdHatePred,cleanPred,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean
0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [49]:
actCancer_final, actClean_final, predCancer_final, predCancerActCancer_final, predClean_final, predCleanActClean_final, predTrueActTrue_final = filter_dataframe(dfFinalFinal)

In [50]:
print_filter(actCancer_final, actClean_final, predCancer_final, predCancerActCancer_final, predClean_final, predCleanActClean_final, predTrueActTrue_final)

Cancer = Data yang mengandung salah satu dari toxic, severe toxic, obscene, threat, insult,  atau identity hate


Total Data di Dataframe = 63978
Banyak data Cancer = 6243
Banyak data Clean = 57735


Banyak Prediction Cancer yang benar = 4
Percentage dari Prediction Cancer yang benar = 0.06407176037161622%


Banyak Prediction Clean yang benar = 57702
Percentage dari Prediction Clean yang benar = 99.94284229670043%


Banyak Prediction yang benar = 57706
Percentage dari Prediction yang benar = 90.19663009159399%
